In [47]:
import numpy as np
import cv2
                    
def applyHomography(img, H):
    dIm = np.array([[0,0,img.shape[0],img.shape[0]],[0, img.shape[1], 0, img.shape[1]], [1, 1, 1, 1]])
    dImNew = np.matmul(H, dIm)
    dImNew[0] = dImNew[0]/dImNew[2]
    dImNew[1] = dImNew[1]/dImNew[2]

    minH = int(np.amin(dImNew[0]))
    maxH = int(np.amax(dImNew[0]))
    minW = int(np.amin(dImNew[1]))
    maxW = int(np.amax(dImNew[1]))
    imNew = np.zeros((maxH - minH, maxW - minW,3))
    invH = np.linalg.inv(H)
   
    for row in range(imNew.shape[0]):
        for col in range(imNew.shape[1]):
            coord = np.matmul(invH,[[row + minH], [col + minW], [1]])
            coord /= coord[2]
            if coord[0]<img.shape[0] and coord[0]>=0 and coord[1]<img.shape[1] and coord[1]>=0:
                imNew[row][col] = img[int(coord[0])][int(coord[1])] 
                
    return imNew

def getHomography(im1Pts, im2Pts):
    px = im1Pts[0, 1]
    py = im1Pts[0, 0]
    qx = im1Pts[1, 1]
    qy = im1Pts[1, 0]
    rx = im1Pts[2, 1]
    ry = im1Pts[2, 0]
    sx = im1Pts[3, 1]
    sy = im1Pts[3, 0]
    
    pxp = im2Pts[0, 1]
    pyp = im2Pts[0, 0]
    qxp = im2Pts[1, 1]
    qyp = im2Pts[1, 0]
    rxp = im2Pts[2, 1]
    ryp = im2Pts[2, 0]
    sxp = im2Pts[3, 1]
    syp = im2Pts[3, 0]
    
    t = np.array([[pxp],[pyp],[qxp],[qyp],[rxp],[ryp],[sxp],[syp]])
    P = np.array([[px, py, 1, 0, 0, 0, -px*pxp, -py*pxp],
                  [0, 0, 0, px, py, 1, -px*pyp, -py*pyp],
                  [qx, qy, 1, 0, 0, 0, -qx*qxp, -qy*qxp],
                  [0, 0, 0, qx, qy, 1, -qx*qyp, -qy*qyp],
                  [rx, ry, 1, 0, 0, 0, -rx*rxp, -ry*rxp],
                  [0, 0, 0, rx, ry, 1, -rx*ryp, -ry*ryp],
                  [sx, sy, 1, 0, 0, 0, -sx*sxp, -sy*sxp],
                  [0, 0, 0, sx, sy, 1, -sx*syp, -sy*syp]])
    pInv = np.linalg.inv(P)
    h = pInv.dot(t)
    H = np.array([[h[0, 0], h[1, 0], h[2, 0]], [h[3, 0], h[4, 0], h[5, 0]], [h[6, 0], h[7, 0], 1]])
    return H

"""im1 = cv2.imread("hw3_Task1_Images/Images/Img1.JPG")
dest1Corners = np.array([[0,0], [75, 0], [0,85], [75,85]])
im1Corners = np.array([[644, 500], [663, 503], [643, 531], [663, 535]])
H1 = getHomography(im1Corners,dest1Corners)
cv2.imwrite('Results/Pt2Pt1.jpg', applyHomography(im1, H1))

im2 = cv2.imread("hw3_Task1_Images/Images/Img2.jpeg")
dest2Corners = np.array([[0,0], [84, 0], [0,74], [84,74]])
im2Corners = np.array([[383, 577], [593, 552], [381, 834], [607, 922]])
H2 = getHomography(im2Corners,dest2Corners)
cv2.imwrite('Results/Pt2Pt2.jpg', applyHomography(im2, H2))

im3 = cv2.imread("hw3_Task1_Images/Images/Img3.JPG")
dest3Corners = np.array([[0,0], [55, 0], [0,36], [55,36]])
im3Corners = np.array([[2063, 700], [2665, 718], [2097, 1479], [2695, 1334]])
H3 = getHomography(im3Corners,dest3Corners)
cv2.imwrite('Results/Pt2Pt3.jpg', applyHomography(im3, H3))"""

im1 = cv2.imread("hw3_Task1_Images/Images/Own1.jpg")
dest1Corners = np.array([[0, 0], [220*2, 0], [0,280*2], [220*2,280*2]])
im1Corners = np.array([[234, 199], [626, 233], [256, 813], [647, 739]])
H1 = getHomography(im1Corners,dest1Corners)
cv2.imwrite('Results/Pt2Pt4.jpg', applyHomography(im1, H1))

"""im2 = cv2.imread("hw3_Task1_Images/Images/Own2.jpg")
dest2Corners = np.array([[0,0], [188, 0], [0, 106], [188,106]])
im2Corners = np.array([[625, 521], [3851, 52], [625, 2418], [3852, 2869]])
H2 = getHomography(im2Corners,dest2Corners)
cv2.imwrite('Results/Pt2Pt5.jpg', applyHomography(im2, H2))"""

'im2 = cv2.imread("hw3_Task1_Images/Images/Own2.jpg")\ndest2Corners = np.array([[0,0], [188, 0], [0, 106], [188,106]])\nim2Corners = np.array([[625, 521], [3851, 52], [625, 2418], [3852, 2869]])\nH2 = getHomography(im2Corners,dest2Corners)\ncv2.imwrite(\'Results/Pt2Pt5.jpg\', applyHomography(im2, H2))'